In [1]:
import pandas as pd
from pandas import DataFrame, Series
import numpy as np
import math
from pandas import read_csv
from pandas import read_excel
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_error      

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor

In [21]:
path = 'Predictive Analysis/Features Data v2/'

dateparse = lambda dates: [pd.datetime.strptime(d, '%Y-%m-%d') for d in dates]
dateparse2 = lambda dates: [pd.datetime.strptime(d, '%m/%d/%Y') for d in dates]
    
usage = read_csv(filepath_or_buffer=(path+'usage_v4_3.csv'), 
                   parse_dates=['Date'], date_parser=dateparse2)

In [22]:
student_count = read_csv(filepath_or_buffer=(path+'students_out_of_school_count_2012_18.csv'), 
                   parse_dates=['Date'], date_parser=dateparse)

df = student_count.merge(usage, on='Date', how='left')
df['day_of_week'] = df.Date.dt.weekday_name
# df['month'] = df.Date.dt.month

In [23]:
# df.Holiday = Series(np.where(df.Holiday.isnull(), 'Regular Day', df.Holiday))
# df.Holiday = Series(np.where(df.Holiday.isnull(), '0', df.Holiday))
# df.Holiday = Series(np.where(df.Holiday == 0, 0, 1))

def to_dummies(df):
    for column in df.columns:         
        if df[column].dtype == np.object:
            df_col = pd.get_dummies(df[column])
            df = pd.concat([df, df_col], axis=1)
            df = df.drop(column, axis=1)
    return df

df.Holiday = Series(np.where(df.Holiday.isnull(), 'Regular', df.Holiday))
df['Week'] = 'Week'
df['Week'] = np.where((df.Date.dt.day > 0) & (df.Date.dt.day < 8), 'Week 1', df.Week )
df['Week'] = np.where((df.Date.dt.day > 7) & (df.Date.dt.day < 15), 'Week 2', df.Week )
df['Week'] = np.where((df.Date.dt.day > 14) & (df.Date.dt.day < 22), 'Week 3', df.Week )
df['Week'] = np.where((df.Date.dt.day > 21) & (df.Date.dt.day < 29), 'Week 4', df.Week )
df['Week'] = np.where((df.Date.dt.day > 28), 'Week 5', df.Week)

# df.month = 'month ' + df.month.astype('str')
# df.day_of_week = 'Weekday ' + df.day_of_week.astype('str')
df = to_dummies(df)

df = df[df.Date.dt.year > 2012]

In [24]:
maintenance = read_csv(filepath_or_buffer=(path+'Maintenance_Predictors_Input_v3.csv'), 
                   parse_dates=['Date'], date_parser=dateparse2)
df = df.merge(maintenance, on='Date', how='left')

In [25]:
holiday_effect = read_csv(filepath_or_buffer=(path+'holiday_effect.csv'), 
                   parse_dates=['Date'], date_parser=dateparse2)
df = df.merge(holiday_effect, on='Date', how='left')

In [26]:
# holiday_effect = read_csv(filepath_or_buffer=(path+'holiday_effect.csv'), 
#                    parse_dates=['Date'], date_parser=dateparse2)
holiday_effect.head()

,Date,Holiday_Decrease_Effect
0,2013-01-01,0
1,2013-01-02,0
2,2013-01-03,0
3,2013-01-04,0
4,2013-01-05,0


In [16]:
df['LY Min Ridership'] = df[['LY Ridership', 'LY2 Ridership']].min(axis=1)
df = df.drop(['LY Ridership', 'LY2 Ridership'], axis=1)

In [29]:
df.to_csv('df_all_predictors.csv', index=False)

In [30]:
# df[df.Date.dt.year > 2012].to_csv('df_2013_17_with_dummies.csv', index=False)

In [29]:
class model_fit:    
    
    def __init__(self, df):
        self.df = df
        import math
        import numpy
        from pandas import Series, DataFrame
        import sklearn
        from sklearn import linear_model
        from sklearn.metrics import mean_squared_error, r2_score
        from sklearn.metrics import mean_absolute_error                
    
    def get_Xy(self, df):
        # Split data into predictor and response
#         return df.iloc[:, 0:-1].drop('Date', axis=1), df.iloc[:, -1]
        return df.drop(['Date', 'Ridership'], axis=1), df.loc[:, 'Ridership']

    def performance(self, y_test, y_pred):
        print '============== Month ' + str(self.month) +' =============='
        # The mean absolute error
        print("Mean absolute error: %.2f"
              % mean_absolute_error(y_test, y_pred))

        # The root mean squared error
        print("Root Mean squared error: %.2f"
              % math.sqrt(mean_squared_error(y_test, y_pred)))

        # Explained variance score: 1 is perfect prediction
        print('Variance score(R2): %.2f' % r2_score(y_test, y_pred))

        # Mean Absolute Percentage Error
        mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
        print 'Mean Absolute Percentage Error: {0:.2f}'.format(mape)
        print '-------------------------------------\n'

    def predict(self, regr=linear_model.Lasso(alpha=10), month=1, train_year=None, test_year=None):
        df = self.df
        self.month = month
        if not train_year:        
            if month in [1, 2, 3, 4, 5, 6, 7, 8]:
                train_year = [2013, 2014, 2015, 2016]
                test_year = [2017]
            elif month == 9:
                train_year = [2013, 2014, 2015, 2016]
                test_year = [2017]
            else:
                train_year = [2013, 2014, 2015]
                test_year = [2016]
                
        # Filtering for that particular month
        df_month = df[df.Date.dt.month == month]

        #Drop days with 0 Riderships
        df_month = df_month.drop(df_month[df_month.Ridership == 0].index)

        #Splitting data for training and testing
        df_train = df_month[df_month.Date.dt.year.isin(train_year)]
        df_test = df_month[df_month.Date.dt.year.isin(test_year)]
        self.df_test = df_test

        #Splitting data into training and testing
        X_train, y_train = self.get_Xy(df_train)
        X_test, y_test = self.get_Xy(df_test)
        self.y_test = y_test

        # Model Fitting andn prediction
        y_pred = regr.fit(X_train, y_train).predict(X_test)
        self.y_pred = y_pred
        
        # Print Performance 
        self.performance(y_test, y_pred)

    def get_pred_df(self):
        # Returns the dataframe of prediction period with Actual predictions and absolute Error
        df_pred = self.df_test
        df_pred = df_pred.reset_index().drop('index', axis=1)
        df_pred['Prediction'] = pd.Series(self.y_pred)
        df_pred['Error'] = (df_pred['Prediction'] - df_pred['Ridership'])
        df_pred['% Error'] = df_pred['Error'] * 100.0 / df_pred['Ridership']
        df_pred['Weekday'] = df_pred.Date.dt.weekday_name
        
        #Filtering only required variables
        df_pred = df_pred.loc[:, [ u'Date', u'Weekday', u'total_student_out_of_school_count', u'Ridership',
                                  u'Prediction', u'Error', u'LY Ridership', u'LY2 Ridership',
                                  u'% Error']]
        return df_pred
    
    def sq_error(self):
        # Return list of square error for each day in the month
        se = list((self.y_pred - self.y_test)**2)
        return se

In [170]:
# final = df.copy()
# final.Holiday = Series(np.where(final.Holiday.isnull(), 'Regular', final.Holiday))

# # Function to make dummy variables
# def to_dummies(df):
#     for column in df.columns:         
#         if df[column].dtype == np.object:
#             df_col = pd.get_dummies(df[column])
#             df = pd.concat([df, df_col], axis=1)
#             df = df.drop(column, axis=1)
#     return df

# final = to_dummies(final)
# final = final.drop('Regular', axis=1)

# # final = final.loc[:, [u'Date', u'total_student_out_of_school_count', u'Statefair',
# #        u'Vikings_Home_Game', u'LY Ridership', u'LY2 Ridership',
# #        u'Black Friday', u'Christmas Day', u'Christmas Eve', u'ChristmasWeek',
# #        u'Columbus Day', u'Easter', u'Eid Al-Fitr', u'Eid Al-Fitr - Day After',
# #        u'Eid al-Adha - Day After', u'Fourth of July', u'Good Friday',
# #        u'Halloween', u'Holiday New Year', u'Labor Day', u'MEA Friday',
# #        u'MEA Saturday', u'MEA Thursday', u'MLK Day', u'Memorial Day',
# #        u'Mpls School Patrol', u"New Year's Day", u"New Year's Eve",
# #        u'Palm Sunday', u"President's Day", u'Regular', u'SpringBreak',
# #        u"St. Patrick's Day", u'Suburban School Patrol Day 1',
# #        u'Suburban School Patrol Day 2', u'Thanksgiving', u'Ridership',]]

In [651]:
# se_list = list()
# for i in range(1, 13):
#     model = model_fit(df)
# #     regr = GradientBoostingRegressor(loss='lad', max_depth=2)
# #     regr = linear_model.LinearRegression()
#     regr = linear_model.Lasso(alpha=30)
# #     regr = MLPRegressor(random_state=42, hidden_layer_sizes=20)
#     model.predict(regr=regr, month=i)
#     se =  model.sq_error()
#     se_list = se_list + se

# print 'RMSE for whole year is: {0}'.format(np.sqrt(np.mean(se_list)))

In [78]:
df_jan = df.loc[:, [u'Date', u'total_student_out_of_school_count',
 u'Ridership', u'LY Ridership', u'LY2 Ridership',
       u'MLK Day', u'Weekday 1', u'Weekday 2', u'Weekday 3', u'Weekday 4', 
                    u'Weekday 5', u'Weekday 6', u'Weekday 7', 
                   u'Week 1', u'Week 2', u'Week 3', u'Week 4', u'Week 5']]

df_feb = df.loc[:, [u'Date', u'total_student_out_of_school_count',
       u'Ridership', u'LY Ridership', u'LY2 Ridership',
       u"President's Day", u'Weekday 1',
       u'Weekday 2', u'Weekday 3', u'Weekday 4', u'Weekday 5', u'Weekday 6',
       u'Weekday 7', u'Week 1', u'Week 2', u'Week 3', u'Week 4', u'Week 5']]

df_march = df.loc[:, [u'Date', u'total_student_out_of_school_count',
                           u'Ridership', u'LY Ridership', u'LY2 Ridership',
       u'Easter', u'Good Friday', u'Palm Sunday', u"St. Patrick's Day",
       u'Weekday 1', u'Weekday 2', u'Weekday 3', u'Weekday 4', u'Weekday 5', u'Weekday 6',
       u'Weekday 7', u'Week 1', u'Week 2', u'Week 3', u'Week 4', u'Week 5']]

df_april = df.loc[:, [u'Date', u'total_student_out_of_school_count', 
                      u'Ridership', u'LY Ridership', u'LY2 Ridership',
                      u'Easter', u'Good Friday', 
                      u'Weekday 1', u'Weekday 2', u'Weekday 3', u'Weekday 4', 
                      u'Weekday 5', u'Weekday 6', u'Weekday 7', 
                      u'Week 1', u'Week 2', u'Week 3', u'Week 4', u'Week 5']]

df_may = df.loc[:, [u'Date', u'total_student_out_of_school_count',
       u'Ridership', u'LY Ridership', u'LY2 Ridership',
       u'Mpls School Patrol', u'Suburban School Patrol Day 1',
       u'Weekday 1', u'Weekday 2', u'Weekday 3', u'Weekday 4', u'Weekday 5', u'Weekday 6',
       u'Weekday 7', u'Week 1', u'Week 2', u'Week 3', u'Week 4', u'Week 5']]

df_june = df.loc[:, [u'Date', u'total_student_out_of_school_count',
                u'Ridership', u'LY Ridership', u'LY2 Ridership',
                   u'Weekday 1', u'Weekday 2', u'Weekday 3', u'Weekday 4', 
                     u'Weekday 5', u'Weekday 6', u'Weekday 7', 
                     u'Week 1', u'Week 2', u'Week 3', u'Week 4', u'Week 5']]

df_july = df.loc[:, [u'Date', u'total_student_out_of_school_count',
       u'Ridership', u'LY Ridership', u'LY2 Ridership',
       u'Eid Al-Fitr - Day After', u'Weekday 1',
       u'Weekday 2', u'Weekday 3', u'Weekday 4', u'Weekday 5', u'Weekday 6',
       u'Weekday 7']]

df_aug = df.loc[:, [u'Date', u'total_student_out_of_school_count',u'Vikings_Home_Game',
       u'Ridership', u'LY Ridership', u'LY2 Ridership',
       u'Weekday 1', u'Weekday 2', u'Weekday 3', u'Weekday 4', u'Weekday 5', u'Weekday 6',
       u'Weekday 7', u'Week 1', u'Week 2', u'Week 3', u'Week 4', u'Week 5']]

df_sept = df.loc[:, [u'Date', u'total_student_out_of_school_count', u'Vikings_Home_Game',
                     u'Ridership', u'LY Ridership', 'LY2 Ridership', 'Labor Day',
                     'Weekday 1', u'Weekday 2', u'Weekday 3', u'Weekday 4', 
                     u'Weekday 5', u'Weekday 6', u'Weekday 7', 
                     u'Week 1', u'Week 2', u'Week 3', u'Week 4', u'Week 5']]

df_oct = df.loc[:, [u'Date', u'LY Ridership', 'LY2 Ridership',
       u'total_student_out_of_school_count', u'Weekday 1', u'Weekday 2',
       u'Weekday 3', u'Weekday 4', u'Weekday 5', u'Weekday 6', u'Weekday 7', u'Ridership',
                   u'Week 1', u'Week 2', u'Week 3', u'Week 4', u'Week 5']]


df_nov = df.loc[:, [u'Date', u'total_student_out_of_school_count',
       u'Ridership', u'LY Ridership', u'LY2 Ridership',
       u'Black Friday',u'Thanksgiving', u'Weekday 1',
       u'Weekday 2', u'Weekday 3', u'Weekday 4', u'Weekday 5', u'Weekday 6',
       u'Weekday 7', u'Week 1', u'Week 2', u'Week 3', u'Week 4', u'Week 5']]


df_dec = df.loc[:, [u'Date', u'total_student_out_of_school_count', 
          u'Ridership', u'LY Ridership', u'LY2 Ridership',
          u'Christmas Day', u'ChristmasWeek', u"New Year's Eve", u'Weekday 1', u'Weekday 2', u'Weekday 3',
           u'Weekday 4', u'Weekday 5', u'Weekday 6', u'Weekday 7',
                    u'Week 1', u'Week 2', u'Week 3', u'Week 4', u'Week 5']]


df_month = {'1': df_jan, '2': df_feb, '3': df_march, '4': df_april, '5': df_may, 
           '6': df_june, '7': df_july, '8': df_aug, '9':df_sept, '10':df_oct,
           '11':df_nov, '12':df_dec}

In [101]:
# DataFrame({'column': df_dec.drop(['Ridership', 'Date'], axis=1).columns, 
#           'coef':regr.coef_})

In [107]:
# for i in range(1, 13):
    
    
#     model = model_fit(df_month[str(i)])

#     regr = linear_model.LinearRegression()

#     # regr = GradientBoostingRegressor(loss='lad', max_depth=3)
#     # regr = DecisionTreeRegressor()
#     # regr = RandomForestRegressor()
#     # regr = AdaBoostRegressor()
#     # # regr = GradientBoostingRegressor()
#     # regr = linear_model.Lasso(alpha=10)
#     # regr = linear_model.Ridge(alpha=10)
#     # regr = MLPRegressor(random_state=42, hidden_layer_sizes=20)

#     model.predict(regr=regr, month=i)

In [30]:
df.columns

Index([u'Date', u'total_student_out_of_school_count', u'Statefair',
       u'Vikings_Home_Game', u'Ridership', u'Black Friday', u'Christmas Day',
       u'ChristmasWeek', u'Columbus Day', u'Easter', u'Eid Al-Fitr',
       u'Eid Al-Fitr - Day After', u'Eid al-Adha - Day After',
       u'Fourth of July', u'Good Friday', u'Halloween', u'Holiday New Year',
       u'Labor Day', u'MEA Friday', u'MEA Saturday', u'MEA Thursday',
       u'MLK Day', u'Memorial Day', u'Mpls School Patrol', u'New Year's Day',
       u'New Year's Eve', u'Palm Sunday', u'President's Day', u'Regular',
       u'SpringBreak', u'St. Patrick's Day', u'Suburban School Patrol Day 1',
       u'Suburban School Patrol Day 2', u'Thanksgiving', u'Friday', u'Monday',
       u'Saturday', u'Sunday', u'Thursday', u'Tuesday', u'Wednesday',
       u'LY Min Ridership'],
      dtype='object')

In [35]:
df_march.isnull().sum()

Date                                 0
total_student_out_of_school_count    0
Ridership                            0
LY Min Ridership                     6
Easter                               0
Good Friday                          0
Palm Sunday                          0
St. Patrick's Day                    0
Friday                               0
Monday                               0
Saturday                             0
Sunday                               0
Thursday                             0
Tuesday                              0
Wednesday                            0
dtype: int64

In [94]:
df_june = df.loc[:, [u'Date', u'total_student_out_of_school_count',
                u'Ridership', u'LY New Ridership',
                    u'Friday', u'Monday',
       u'Saturday', u'Sunday', u'Thursday', u'Tuesday', u'Wednesday']]

In [112]:
df_june = df_june[~df_june['LY New Ridership'].isnull()]

In [110]:
df[~df.Date.isin(df_june[df_june['LY New Ridership'].isnull()].Date)]

,Date,total_student_out_of_school_count,Statefair,Vikings_Home_Game,Ridership,LY Ridership,LY2 Ridership,Black Friday,Christmas Day,ChristmasWeek,...,Suburban School Patrol Day 2,Thanksgiving,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,LY New Ridership
212,2013-04-30,35717,0.0,0.0,5365.0,NaN,NaN,0,0,0,...,0,0,0,0,0,0,0,1,0,NaN
219,2013-05-07,35717,0.0,0.0,22139.0,NaN,NaN,0,0,0,...,0,0,0,0,0,0,0,1,0,NaN
233,2013-05-21,35717,0.0,0.0,12253.0,NaN,NaN,0,0,0,...,0,0,0,0,0,0,0,1,0,NaN
251,2013-06-08,436015,0.0,0.0,37825.0,NaN,NaN,0,0,0,...,0,0,0,0,1,0,0,0,0,NaN
332,2013-08-28,346731,1.0,0.0,24186.0,NaN,NaN,0,0,0,...,0,0,0,0,0,0,0,0,1,NaN
386,2013-10-21,0,0.0,0.0,6632.0,NaN,NaN,0,0,0,...,0,0,0,1,0,0,0,0,0,NaN


In [131]:
df['LY New Ridership'] = df[['LY Ridership', 'LY2 Ridership']].min(axis=1)

In [125]:
df_june.isnull().sum()

Date                                    0
total_student_out_of_school_count       0
Ridership                               0
LY New Ridership                        0
Friday                                  0
Saturday                                0
Sunday                                  0
Thursday                                0
Tuesday                                 0
Wednesday                               0
Week 1                               1738
Week 2                               1738
Week 3                               1738
Week 4                               1738
Week 5                               1738
dtype: int64

In [139]:
dateparse = lambda dates: [pd.datetime.strptime(d, '%Y-%m-%d') for d in dates]
dateparse2 = lambda dates: [pd.datetime.strptime(d, '%m/%d/%Y') for d in dates]
    
maintenance = read_csv(filepath_or_buffer=(path+'Maintenance_Predictors_Input_v3.csv'), 
                   parse_dates=['Date'], date_parser=dateparse2)

In [143]:
df_main = df.merge(maintenance, on='Date', how='left')

In [144]:
df_main.columns

Index([u'Date', u'total_student_out_of_school_count', u'Statefair',
       u'Vikings_Home_Game', u'Ridership', u'LY Ridership', u'LY2 Ridership',
       u'Black Friday', u'Christmas Day', u'ChristmasWeek', u'Columbus Day',
       u'Easter', u'Eid Al-Fitr', u'Eid Al-Fitr - Day After',
       u'Eid al-Adha - Day After', u'Fourth of July', u'Good Friday',
       u'Halloween', u'Holiday New Year', u'Labor Day', u'MEA Friday',
       u'MEA Saturday', u'MEA Thursday', u'MLK Day', u'Memorial Day',
       u'Mpls School Patrol', u'New Year's Day', u'New Year's Eve',
       u'Palm Sunday', u'President's Day', u'Regular', u'SpringBreak',
       u'St. Patrick's Day', u'Suburban School Patrol Day 1',
       u'Suburban School Patrol Day 2', u'Thanksgiving', u'Friday', u'Monday',
       u'Saturday', u'Sunday', u'Thursday', u'Tuesday', u'Wednesday',
       u'Week 1', u'Week 2', u'Week 3', u'Week 4', u'Week 5',
       u'LY New Ridership', u'3', u'7', u'8', u'9', u'10', u'11', u'16', u'19',
       u'23'

In [174]:
df_main['LY New Ridership'] = df_main[['LY Ridership', 'LY2 Ridership']].min(axis=1)

df_june = df_main.loc[:, [u'Date', u'total_student_out_of_school_count',
                u'Ridership', u'LY New Ridership',
                    u'Friday',
       u'Saturday', u'Sunday', u'Thursday', u'Tuesday', u'Wednesday']]

df_june = df_june[~df_june['LY New Ridership'].isnull()]
model = model_fit(df_june)
# regr = linear_model.LinearRegression()

# regr = GradientBoostingRegressor(loss='lad', max_depth=3)
# regr = DecisionTreeRegressor()
# regr = RandomForestRegressor()
# regr = AdaBoostRegressor()
# # regr = GradientBoostingRegressor()
regr = linear_model.Lasso(alpha=20)
# regr = linear_model.Ridge(alpha=10)
# regr = MLPRegressor(random_state=42, hidden_layer_sizes=20)

model.predict(regr=regr, month=6)
model.get_pred_df()

============== Month 6 ==============
Mean absolute error: 4931.41
Root Mean squared error: 5500.48
Variance score(R2): 0.40
Mean Absolute Percentage Error: 23.80
-------------------------------------



,Date,Weekday,total_student_out_of_school_count,Ridership,Prediction,Error,LY Ridership,LY2 Ridership,% Error
0,2017-06-01,Thursday,32186,9529.0,20574.387375,11045.387375,NaN,NaN,115.913395
1,2017-06-02,Friday,56612,21030.0,25437.408346,4407.408346,NaN,NaN,20.957719
2,2017-06-03,Saturday,436015,30256.0,37874.581319,7618.581319,NaN,NaN,25.180398
3,2017-06-04,Sunday,436015,16486.0,22094.644166,5608.644166,NaN,NaN,34.020649
4,2017-06-05,Monday,56612,13175.0,20581.575456,7406.575456,NaN,NaN,56.216892
5,2017-06-06,Tuesday,56612,15936.0,20265.285040,4329.285040,NaN,NaN,27.166698
6,2017-06-07,Wednesday,91723,17401.0,19423.833883,2022.833883,NaN,NaN,11.624814
7,2017-06-08,Thursday,315681,19839.0,25941.356150,6102.356150,NaN,NaN,30.759394
8,2017-06-09,Friday,400298,24237.0,30013.103345,5776.103345,NaN,NaN,23.831759
9,2017-06-10,Saturday,436015,31380.0,37439.789090,6059.789090,NaN,NaN,19.310991
